In [1]:
import os
import random
from glob import glob
import numpy as np
from PIL import Image
from sklearn.metrics import confusion_matrix
from skimage.metrics import structural_similarity as ssim
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
import cv2
import copy
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Paths to dataset images and ground truths
dataset_path = r'CHASEDB1'
image_files = glob(os.path.join(dataset_path, '*.jpg'))
ground_truth_files = glob(os.path.join(dataset_path, '*_1stHO.png'))
image_files.sort()
ground_truth_files.sort()

In [3]:
def accuracy_func(image1, image2):
    segmented_vessels = image1
    segmented_image = image2

    ssim_score = ssim(segmented_image, segmented_vessels, data_range=255)
    mse = np.mean((image1 - image2) ** 2)
    psnr_score = 20 * np.log10(255.0 / np.sqrt(mse)) if mse != 0 else float('inf')

    generated_binary = (segmented_vessels > 0).astype(np.uint8)
    target_binary = (segmented_image > 0).astype(np.uint8)

    tn, fp, fn, tp = confusion_matrix(target_binary.ravel(), generated_binary.ravel()).ravel()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    f1 = 2 / ((1 / specificity) + (1 / sensitivity))
    
    return accuracy, sensitivity, specificity, f1, ssim_score, psnr_score

In [4]:
def threshold(img,k):
    ret = copy.deepcopy(img)
    ret[ret<k] = 0
    ret[ret>=k] = 255
    return ret

def GlobalOtsu(img):
    foreground = img[img>=0]
    background = img[img<0]
    
    final_var = (np.var(foreground) * len(foreground) + np.var(background) * len(background))/(len(foreground) + len(background))
    if(np.isnan(final_var)):
        final_var = -1
        
    final_thresh = 0
    for i in np.linspace(np.min(img), np.max(img), num=255):
        foreground = img[img>=i]
        background = img[img<i]
        var = (np.var(foreground) * len(foreground) + np.var(background) * len(background))/(len(foreground) + len(background))
        
        if(np.isnan(var)):
            var = -1
            
        if(var!=-1 and (var<final_var or final_var ==-1)):
            final_var = var
            final_thresh = i
    return threshold(img,final_thresh)

In [7]:
metrics_without_clahe = []
metrics_with_clahe = []

selected_indices = random.sample(range(len(image_files)), 10)

# Process all images in the dataset
for idx in selected_indices:
    img_path = image_files[idx]
    ground_img_path = ground_truth_files[idx]

    # Load images
    img = Image.open(img_path)
    ground_img = Image.open(ground_img_path)

    img_array = np.array(img)
    ground_img_array = np.array(ground_img)

    # Extract Green Channel
    green_channel = img_array[:, :, 1]

    # Apply Random CLAHE with Varying Clip Limit
    random_clip_limit = random.uniform(2, 5)  # Randomly select clip limit
    clahe = cv2.createCLAHE(clipLimit=random_clip_limit, tileGridSize=(8, 8))
    cl_img_green = clahe.apply(green_channel)

    # Morphological Operations
    cell_disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

    # Without CLAHE
    Topen = cv2.morphologyEx(green_channel, cv2.MORPH_OPEN, cell_disc)
    Tclose = cv2.morphologyEx(Topen, cv2.MORPH_CLOSE, cell_disc)
    TopHat = green_channel - Tclose
    min_image = cv2.dilate(cv2.erode(TopHat, kernel), kernel)
    HessWide = hessian_matrix(min_image, sigma=4, order='rc')
    EignWide = hessian_matrix_eigvals(HessWide)[1]
    val = GlobalOtsu(1 - EignWide)

    # With CLAHE
    Topen_clahe = cv2.morphologyEx(cl_img_green, cv2.MORPH_OPEN, cell_disc)
    Tclose_clahe = cv2.morphologyEx(Topen_clahe, cv2.MORPH_CLOSE, cell_disc)
    TopHat_clahe = cl_img_green - Tclose_clahe
    min_image_clahe = cv2.dilate(cv2.erode(TopHat_clahe, kernel), kernel)
    HessWide_clahe = hessian_matrix(min_image_clahe, sigma=4, order='rc')
    EignWide_clahe = hessian_matrix_eigvals(HessWide_clahe)[1]
    val_clahe = GlobalOtsu(1 - EignWide_clahe)

    # Compute Metrics
    metrics_without_clahe.append(accuracy_func(ground_img_array, val))
    metrics_with_clahe.append(accuracy_func(ground_img_array, val_clahe))

# Calculate Average Metrics
metrics_without_clahe = np.mean(metrics_without_clahe, axis=0)
metrics_with_clahe = np.mean(metrics_with_clahe, axis=0)

# Report Metrics
print("Average Metrics Without CLAHE:")
print(f"Accuracy: {metrics_without_clahe[0]:.4f}, Sensitivity: {metrics_without_clahe[1]:.4f}, Specificity: {metrics_without_clahe[2]:.4f}, F1-Score: {metrics_without_clahe[3]:.4f}, SSIM: {metrics_without_clahe[4]:.4f}, PSNR: {metrics_without_clahe[5]:.4f}")

print("\nAverage Metrics With Random CLAHE:")
print(f"Accuracy: {metrics_with_clahe[0]:.4f}, Sensitivity: {metrics_with_clahe[1]:.4f}, Specificity: {metrics_with_clahe[2]:.4f}, F1-Score: {metrics_with_clahe[3]:.4f}, SSIM: {metrics_with_clahe[4]:.4f}, PSNR: {metrics_with_clahe[5]:.4f}")

Average Metrics Without CLAHE:
Accuracy: 0.9280, Sensitivity: 0.5071, Specificity: 0.9614, F1-Score: 0.6607, SSIM: 0.7971, PSNR: 11.4131

Average Metrics With Random CLAHE:
Accuracy: 0.9145, Sensitivity: 0.4352, Specificity: 0.9641, F1-Score: 0.5958, SSIM: 0.7404, PSNR: 10.3739
